# Visualizer for Original Lesion Segmentations

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from ipywidgets import interactive, widgets
from IPython.display import display

In [ ]:
# Function to plot an image from a list at a specified index
def plot_image(list_of_images, list_of_masks, image_idx, mask_transparency):
    original_image = np.rot90(list_of_images[image_idx], k=3)
    mask = np.rot90(list_of_masks[image_idx], k=3)

    # Overlay the mask on the original image
    masked_image = np.ma.masked_where(mask == 0, original_image)
    
    plt.imshow(original_image, cmap='gray')
    plt.imshow(masked_image, cmap='jet', alpha=mask_transparency)  # Adjust alpha for transparency
    plt.axis('off')  # Turn off axis numbers
    plt.show()

# Function to get a transparency scale widget
def get_transparency_scale():
    return widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.1,
        description='Mask Transparency:',
        continuous_update=False
    )

# Function to get a slider widget
def get_slider(max_value):
    return widgets.IntSlider(
        value=0,
        min=0,
        max=max_value,
        step=1,
        description='Image Index:',
        continuous_update=False
    )

# Wrapper function to create an interactive plot with a list of images
def create_interactive_plot(list_of_images, list_of_masks, title):
    slider = get_slider(len(list_of_images) - 1)
    transparency_scale = get_transparency_scale()
    interactive_plot = interactive(plot_image, list_of_images=widgets.fixed(list_of_images), list_of_masks=widgets.fixed(list_of_masks), 
                                    mask_transparency=transparency_scale, image_idx=slider)
    # Display the title as a text above the plot
    display(widgets.Label(value=f'\n{title}', style={'description_width': 'initial'}))
    display(interactive_plot)

def plot_interactive_plot(original_path, mask_path, patient_number, is_cropped):
    
    # Load the common mask file
    common_mask_image = nib.load(mask_path).get_fdata()

    original_image = nib.load(original_path).get_fdata()
    original_images_to_slide_over = [original_image[:, :, i] for i in range(original_image.shape[2])]

    # Use the common mask for all original images
    mask_images_to_slide_over = [common_mask_image[:, :, i] for i in range(common_mask_image.shape[2])]

    create_interactive_plot(original_images_to_slide_over, mask_images_to_slide_over, "T2W")


Private dataset lesion segmentations can be visualized by providing patient id.

In [ ]:
patient = "001"
original_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/nnUNet_raw/Dataset777_ProstateLesion/imagesTs/{patient}_{patient}_0000.nii.gz'
mask_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Dataset_Segmentation_Results/{patient}_{patient}.nii.gz'
plot_interactive_plot(original_path, mask_path, patient, is_cropped=False)